In [ ]:
from matplotlib import pyplot as plt
from graph_cnn.graph import create_final_graph
import networkx as nx
from graph_cnn.model import create_model 
import tensorflow as tf
from graph_cnn.graph import mutate
from graph_cnn.individual import Individual

In [ ]:
g1 = create_final_graph(5,0.8)
nx.draw(g1, with_labels=True)
plt.draw()

In [ ]:
nx.draw(g1, with_labels=True)
plt.draw()
g1.edges(data=True)

In [ ]:
g1.nodes

In [ ]:
'''import numpy as np
import random
import tensorflow_datasets as tfds

#import minst dataset and split to test trin
mnist_dataset, mnist_info = tfds.load(name='crema_d', with_info=True, as_supervised=True)

def scale(waveform, label):
    waveform = tf.cast(waveform,tf.float64)
    spectrogram = tf.signal.stft(
    waveform, frame_length=255, frame_step=28)
    spectrogram = tf.abs(spectrogram)
    spectrogram = spectrogram[..., tf.newaxis]
    spectrogram = tf.image.resize(spectrogram,(28,28))
    label = tf.one_hot(label, 6)
    return spectrogram, label

train_data = mnist_dataset['train'].map(scale)
test_data = mnist_dataset['test'].map(scale)
train_image , train_label = next(iter(train_data.batch(1)))

train_ds = train_data.shuffle(10).cache().prefetch(10).batch(64)
test_ds = test_data.cache().prefetch(10).batch(64)

print(train_image.shape)
print(train_label.shape)'''

In [ ]:
import numpy as np
import random
import tensorflow_datasets as tfds

mnist_dataset , mnist_info = tfds.load(name='mnist', with_info=True, as_supervised=True,split=['train', 'test[:50%]'],shuffle_files=True)
train_data = mnist_dataset[0]
test_data = mnist_dataset[1]
def scale(image, label):
    image = tf.cast(image,tf.float64)
    image = tf.image.resize(image,(28,28))
    image = image/225
    label = tf.one_hot(label, 10)
    return image, label
train_data = train_data.map(scale)
test_data = test_data.map(scale)
train_ds = train_data.cache().batch(64).prefetch(tf.data.experimental.AUTOTUNE)
test_ds = test_data.cache().batch(64).prefetch(tf.data.experimental.AUTOTUNE)

In [ ]:
'''!pip install -q opendatasets

import opendatasets as od
import pandas as pd

od.download('https://www.kaggle.com/datasets/puneet6060/intel-image-classification')
'''

In [ ]:
'''import tensorflow as tf
import os
import glob
import numpy as np
import random

import tensorflow as tf
import os
import glob
import numpy as np

class CustomImageDataGenerator(tf.keras.utils.Sequence):
    def __init__(self, directory, batch_size=28, image_size=(128, 128), shuffle=True):
        self.directory = directory
        self.batch_size = batch_size
        self.image_size = image_size
        self.shuffle = shuffle
        self.filepaths = self._get_filepaths()
        self.indexes = np.arange(len(self.filepaths))
        self.class_mapping = self._create_class_mapping()
        if self.shuffle:
            np.random.shuffle(self.indexes)

    def _get_filepaths(self):
        pattern = os.path.join(self.directory, '*/*.jpg')  # Assuming PNG format, modify as needed
        filepaths = glob.glob(pattern)
        random.shuffle(filepaths)
        return filepaths

    def _create_class_mapping(self):
        classes = sorted(set(os.listdir(self.directory)))
        print(len(classes))
        class_mapping = {cls: idx for idx, cls in enumerate(classes)}
        return class_mapping

    def _load_and_preprocess_image(self, filepath):
        image = tf.keras.preprocessing.image.load_img(filepath, target_size=self.image_size, color_mode='grayscale')
        image = tf.keras.preprocessing.image.img_to_array(image)
        datagen = tf.keras.preprocessing.image.ImageDataGenerator(
        width_shift_range=0.2,
        height_shift_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        vertical_flip=True
        )
        image = image.reshape((1,) + image.shape)
        augmented_image = datagen.flow(image).next()[0]
        image = augmented_image / 255.0  # Normalize to [0, 1]
        return image

    def __len__(self):
        return int(np.ceil(len(self.filepaths) / self.batch_size))

    def _downsample(self, batch_images, batch_labels):
        unique_labels, label_counts = np.unique(batch_labels.argmax(axis=1), return_counts=True)
        min_label_count = np.min(label_counts)

        downsampled_images = []
        downsampled_labels = []

        for label in unique_labels:
            label_indices = np.where(batch_labels.argmax(axis=1) == label)[0]
            selected_indices = np.random.choice(label_indices, size=min_label_count, replace=False)

            downsampled_images.extend(batch_images[selected_indices])
            downsampled_labels.extend(batch_labels[selected_indices])

        downsampled_images = np.array(downsampled_images)
        downsampled_labels = np.array(downsampled_labels)

        # Shuffle the downsampled data
        shuffle_indices = np.arange(len(downsampled_labels))
        np.random.shuffle(shuffle_indices)

        return downsampled_images[shuffle_indices], downsampled_labels[shuffle_indices]

    def __getitem__(self, index):
        batch_filepaths = self.filepaths[index * self.batch_size:(index + 1) * self.batch_size]
        batch_images = []
        batch_labels = []

        for filepath in batch_filepaths:
            image = self._load_and_preprocess_image(filepath)
            label_str = os.path.basename(os.path.dirname(filepath))
            label = self.class_mapping[label_str]
            label = tf.keras.utils.to_categorical(label,6)

            batch_images.append(image)
            batch_labels.append(label)

        batch_images = np.array(batch_images)
        batch_labels = np.array(batch_labels)
        
        batch_images, batch_labels = self._downsample(batch_images, batch_labels)

        return batch_images, batch_labels

    def on_epoch_end(self):
        if self.shuffle:
            np.random.shuffle(self.indexes)

# Example usage:
train_generator = CustomImageDataGenerator(directory='intel-image-classification/seg_train/seg_train', batch_size=512)
test_generator = CustomImageDataGenerator(directory='intel-image-classification/seg_test/seg_test', batch_size=512)

# Rest of the code remains the same...


train_ds = tf.data.Dataset.from_generator(
    generator=lambda: train_generator,
    output_signature=(
        tf.TensorSpec(shape=(None,128,128,1), dtype=tf.float28),
        tf.TensorSpec(shape=(None,6), dtype=tf.float28),
    )
).cache()

test_ds = tf.data.Dataset.from_generator(
    generator=lambda: test_generator,
    output_signature=(
        tf.TensorSpec(shape=(None,128,128,1), dtype=tf.float28),
        tf.TensorSpec(shape=(None,6), dtype=tf.float28),
    )
).cache()

for epoch in range(5):
    for batch_x, batch_y in test_ds:
        print(batch_x.shape)
        print(batch_y.shape)
        print(batch_y)
        break'''

In [ ]:
model = create_model(g1,(28,28,1),10,include_aux=True)

In [ ]:

model.compile(optimizer=tf.keras.optimizers.SGD(0.001), loss=tf.keras.losses.CategoricalCrossentropy(), metrics=[tf.keras.metrics.CategoricalAccuracy(),
                                                                                                               tf.keras.metrics.AUC(from_logits=True),
                                                                                                               #tf.keras.metrics.Precision(),
                                                                                                               #tf.keras.metrics.Recall()
                                                                                                               ])

In [ ]:
model.summary()

In [ ]:
tf.keras.utils.plot_model(model, to_file='model.png', show_shapes=True)

In [ ]:
'''g1 = create_random_graph(5,0.5)
g1 = assign_states(g1,(1,3,1))
fig , ax = plt.subplots(2,1)
g3 = mutate_dag(g1)
nx.draw(g1, with_labels=True, ax=ax[0])
nx.draw(g3, with_labels=True, ax=ax[1])
plt.draw()'''


In [ ]:
'''g4 = crossover(g1,g3)
fig , ax = plt.subplots(2,1)
nx.draw(g1, with_labels=True, ax=ax[0])
nx.draw(g4, with_labels=True, ax=ax[1])
plt.draw()'''

In [15]:
model.fit(train_ds, epochs=10, validation_data=test_ds)

141/938 [===>..........................] - ETA: 14:32 - loss: 2.3412 - categorical_accuracy: 0.1022 - auc: 0.4945

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix
from IPython.display import clear_output

class ClassificationMatrixCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        clear_output(wait=True)
        y_pred =  np.array(self.model.predict(test_ds))
        print(y_pred.shape)
        if len(y_pred.shape) == 3:
            y_pred = y_pred[0]
        predicted_labels = np.argmax(y_pred, axis=1)
        true_labels = np.concatenate([y for x, y in test_ds], axis=0)
        print(true_labels.shape,predicted_labels.shape)
        true_labels = np.argmax(true_labels,axis=-1)
        cm = confusion_matrix(true_labels, predicted_labels)
        print(cm)

In [ ]:
optimizer = tf.keras.optimizers.legacy.Adam(learning_rate=0.001)

class_names = ['angry', 'disgust', 'fear', 'happy', 'sad', 'surprise', 'neutral']
class_names = ['0','1','2','3','4','5','6','7','8','9']
callback = [ClassificationMatrixCallback(),
            #tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3),
            tf.keras.callbacks.ReduceLROnPlateau(monitor='loss', factor=0.1, patience=5),
            tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=10),]
metrics = [tf.keras.metrics.AUC(from_logits=True),tf.keras.metrics.CategoricalAccuracy(),tf.keras.metrics.Precision(),tf.keras.metrics.Recall(),tf.keras.metrics.AUC(name='prc', curve='PR')]
model.compile(optimizer=optimizer, loss=tf.keras.losses.CategoricalCrossentropy(),metrics=[
                                                                                            tf.keras.metrics.Accuracy(),
                                                                                            tf.keras.metrics.AUC(from_logits=True),
                                                                                            tf.keras.metrics.CategoricalAccuracy(),
                                                                                            #tf.keras.metrics.Precision(),
                                                                                            #tf.keras.metrics.Recall(),
                                                                                            tf.keras.metrics.AUC(name='prc', curve='PR'),
                                                                                            ]
                                                                                            )
model.summary()
class_names = ['0','1','2','3','4','5','6','7','8','9']

In [ ]:
tf.keras.utils.plot_model(model, to_file='model.png', show_shapes=True)

In [ ]:
model.fit(train_ds,validation_data=test_ds, epochs=30,
          #steps_per_epoch=50,
           callbacks=[callback],)

In [ ]:
loss = tf.keras.losses.CategoricalCrossentropy()
from graph_cnn.generation.generate import Generation
ga = Generation(input_size=(128,128,1),output_size=(102),nodes=13
,edges=0.8,population=17,limit=18,train_ds=train_ds,test_ds=test_ds,optimizer=optimizer,loss=loss,metrics=metrics,callbacks=callback)
ga.run(10,0.8,0.8)